In [24]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import CCA
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

In [2]:
#11/5: cca is unsup. for expert: fit in X_expert, X_other instead of y



#10/29. do PCA instead of CCA without expert user. should take same inputs. figure out what structure needed to effectively run PCA
##### try LDA or SVC instead of KNN 





#10/22
#do fft on data prior to working with it
#fft: time to freq
#ifft: freq to time
#where they have EMG, it should be each gesture indiv. pull in the data as the dataframe with each column
#look at kai's github for breaking down pickle into gestures

#check the directory structure of momona's code

#time data going in the moments and time converted to freq should have the same moments.

In [3]:
stand_path = 'C:\\Users\\User\\Downloads\\Momona Data\\NEWmetadata_EMG_standardized_allusers.pkl'
pers_path = 'C:\\Users\\User\\Downloads\\Momona Data\\NEWmetadata_EMG_userdef_allusers.pkl'

# Open the file in binary mode and load the data
with open(pers_path, 'rb') as file:
    EMG_pers = pickle.load(file)
    
with open(stand_path, 'rb') as file:
    EMG_stand = pickle.load(file)

# Now 'data' contains the deserialized Python object
print(EMG_pers)

      Participant Gesture_ID Gesture_Num      EMG1      EMG2      EMG3  \
0            P102  gesture-1           1  0.000002  0.000001  0.000001   
1            P102  gesture-1           1  0.000002  0.000002  0.000001   
2            P102  gesture-1           1  0.000002  0.000002  0.000002   
3            P102  gesture-1           1  0.000002  0.000001  0.000002   
4            P102  gesture-1           1  0.000002  0.000001  0.000001   
...           ...        ...         ...       ...       ...       ...   
99195        P011  gesture-5          10  0.000016  0.000003  0.000003   
99196        P011  gesture-5          10  0.000014  0.000003  0.000003   
99197        P011  gesture-5          10  0.000014  0.000003  0.000003   
99198        P011  gesture-5          10  0.000014  0.000003  0.000002   
99199        P011  gesture-5          10  0.000016  0.000003  0.000002   

           EMG4      EMG5      EMG6      EMG7      EMG8      EMG9     EMG10  \
0      0.000001  0.000003  0.000

# Feature Engineering

In [4]:
def zero_order(df_freq):
    zero_order_moments_log = []
    zero_order_moments_raw = []

    for sensor in df_freq.columns:
        # Extract the frequency domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        
        # Step 1: Square the signal (power) at each frequency
        signal_squared = np.abs(time_data) ** 2
        
        # Step 2: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 3: Take the logarithm of the total power
        log_total_power = np.log(total_power)
        
        # Store the result in the dictionary
        zero_order_moments_log.append(log_total_power)
        zero_order_moments_raw.append(total_power)
        
    return zero_order_moments_log, zero_order_moments_raw


In [5]:
def first_order(df_freq, zero_order_raw):
    # Initialize lists to store the results for each sensor
    first_order_moments_log = []
    first_order_moments_raw = []
    
    for i,sensor in enumerate(df_freq.columns):
        # Extract the frequency domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        
        # Step 1: Take first derivative
        first_deriv = np.gradient(time_data)
        
        # Step 2: Square the signal (power) at each frequency of the first deriv
        signal_squared = np.abs(first_deriv) ** 2
        
        # Step 3: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 4: Take the logarithm of the total power
        log_total_power = np.log(total_power / (zero_order_raw[i]**2))
        
        # Store the results in the lists
        first_order_moments_log.append(log_total_power)
        first_order_moments_raw.append(total_power)

    # Convert lists to numpy arrays for consistency
    first_order_moments_log = np.array(first_order_moments_log)
    first_order_moments_raw = np.array(first_order_moments_raw)
    
    return first_order_moments_log, first_order_moments_raw


In [6]:
def second_order(df_freq, zero_order_raw):
    second_order_log = []
    second_order_raw = []

    for i, sensor in enumerate(df_freq.columns):
        # Extract the time domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        
        # Step 1: Take first derivative
        first_deriv = np.gradient(time_data)
        
        #Step 2: take second derivative
        second_deriv = np.gradient(first_deriv)
        
        # Step 3: Square the signal (power) at each frequency of the first deriv
        signal_squared = np.abs(second_deriv) ** 2
        
        # Step 4: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 4: Take the logarithm of the total power
        log_total_power = np.log(total_power/(zero_order_raw[i]**4))
        
        # Store the result in the dictionary
        second_order_log.append(log_total_power)
        second_order_raw.append(total_power)

    
    return second_order_log, second_order_raw


In [7]:
#working version of third_order that has precautions so no neg values inside log
def third_order(second_order_raw, first_order_raw, zero_order_raw):
    second_order_raw = np.array(second_order_raw)
    first_order_raw = np.array(first_order_raw)
    zero_order_raw = np.array(zero_order_raw)
    
    # Step 1: Compute the square roots (ensure no negative values before sqrt)
    sqrt_first_diff = np.sqrt(np.maximum(zero_order_raw - first_order_raw, 1e-10))  # Handle small or negative values
    sqrt_second_diff = np.sqrt(np.maximum(zero_order_raw - second_order_raw, 1e-10))  # Handle small or negative values
    
    # Step 2: Perform the dot product (ensure no division by zero by adding a small constant)
    dot_product = np.dot(sqrt_first_diff, sqrt_second_diff)
    dot_product = max(dot_product, 1e-10)  # Avoid division by zero or extremely small numbers
    
    # Step 3: Compute the sparseness ratio (ensure the result is positive)
    sparseness = zero_order_raw / dot_product
    
    # Step 4: Logarithm of the sparseness (ensure no negative or zero values inside the log)
    # We use np.maximum to ensure all values are at least 1e-10 to avoid taking log of non-positive values.
    safe_sparseness = np.maximum(sparseness, 1e-10)
    
    #print("Safe sparseness:", safe_sparseness)
    
    # Step 5: Apply the logarithm for third-order moments
    third_order_moments_log = np.log(safe_sparseness)
    
    return third_order_moments_log


In [8]:
def fourth_order(df_freq, zero_order_raw, first_order_raw, second_order_raw):
    # Initialize a list to store the fourth-order moments log for each sensor
    fourth_order_moments_logs = []
    
    for i,sensor in enumerate(df_freq.columns):
        # Extract the time domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        
        # Step 1: Take first derivative
        first_deriv = np.gradient(time_data)
        
        # Step 2: Absolute value of the signal (power) at each frequency of the first derivative
        signal_abs = np.abs(first_deriv)
        
        # Step 3: Integrate (sum) the absolute values to get the total power
        total_power = np.sum(signal_abs)
        
        # Step 4: Compute the fourth-order moments log for this sensor
        moment_log = np.log(np.sqrt((first_order_raw[i]**2) / 
                           (zero_order_raw[i] * second_order_raw[i])) / total_power)
        
        # Store the result in the list
        fourth_order_moments_logs.append(moment_log)
    
    # Convert list to numpy array for consistency
    fourth_order_moments_logs = np.array(fourth_order_moments_logs)
    
    return fourth_order_moments_logs


In [9]:
def create_feature_vectors(group):
    result_vector = []
    #can only run features on EMG columns
    emg_columns = [col for col in group.columns if col.startswith('EMG')]
    
    for emg_col in emg_columns:
        data = group[[emg_col]] #run on EMG columns. convert to df with single column to run with all the features
        
        #zero-order
        zero_order_log, zero_order_raw = zero_order(data)
        result_vector.append(zero_order_log)
        
        #first-order
        first_order_log, first_order_raw = first_order(data, zero_order_raw)
        result_vector.append(first_order_log)
        
        #second-order
        second_order_log, second_order_raw = second_order(data, zero_order_raw)
        result_vector.append(second_order_log)
        
        #third-order
        third_order_log = third_order(second_order_raw, first_order_raw, zero_order_raw)
        result_vector.append(third_order_log)
        
        #fourth-order
        fourth_order_log = fourth_order(data, zero_order_raw, first_order_raw, second_order_raw)
        result_vector.append(fourth_order_log)
        
    return pd.DataFrame({
        'Participant': [group['Participant'].iloc[0]],
        'Gesture_ID': [group['Gesture_ID'].iloc[0]],
        'Gesture_Num': [group['Gesture_Num'].iloc[0]],
        'feature': [np.array(result_vector)]
    })
result = EMG_stand.groupby(['Participant', 'Gesture_ID', 'Gesture_Num']).apply(create_feature_vectors)

#output is df with particpant, gesture_ID, gesture_num and feature (holds 80 len vector)
result = result.reset_index(drop=True)

In [10]:
userdef = EMG_pers.groupby(['Participant', 'Gesture_ID', 'Gesture_Num']).apply(create_feature_vectors)

#output is df with particpant, gesture_ID, gesture_num and feature (holds 80 len vector)
userdef = userdef.reset_index(drop=True)

In [11]:
#confirmed that all vectors are the same length (80)
for i in range(40):
    print(len(userdef['feature'].loc[i]))

80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80


In [12]:
userdef

,Participant,Gesture_ID,Gesture_Num,feature
0,P004,gesture-1,1,"[[-23.26249377989712], [18.016590958660785], [..."
1,P004,gesture-1,10,"[[-23.24165786128314], [17.846433634527166], [..."
2,P004,gesture-1,2,"[[-23.23998758618434], [16.66865598878668], [6..."
3,P004,gesture-1,3,"[[-23.207208838399126], [17.61917815123758], [..."
4,P004,gesture-1,4,"[[-23.284384985499166], [17.674295511346255], ..."
...,...,...,...,...
1545,P132,gesture-5,5,"[[-17.55741253047571], [15.749595268750634], [..."
1546,P132,gesture-5,6,"[[-17.040444837807332], [15.687440605336647], ..."
1547,P132,gesture-5,7,"[[-17.29138625623744], [15.337172953351514], [..."
1548,P132,gesture-5,8,"[[-17.134138008109343], [15.183812380351172], ..."


# CCA

In [13]:
#convert Gesture_ID to numerical with new Gesture_Encoded column

label_encoder = LabelEncoder()
result['Gesture_Encoded'] = label_encoder.fit_transform(result['Gesture_ID'])
userdef['Gesture_Encoded'] = label_encoder.fit_transform(userdef['Gesture_ID'])

In [14]:
#count number of rows per PID
result['Participant'].value_counts()
userdef['Participant'].value_counts() #yay! all user-def are same length


P004    50
P112    50
P128    50
P127    50
P126    50
P125    50
P124    50
P123    50
P122    50
P121    50
P119    50
P118    50
P116    50
P115    50
P114    50
P111    50
P005    50
P110    50
P109    50
P108    50
P107    50
P106    50
P105    50
P104    50
P103    50
P102    50
P011    50
P010    50
P008    50
P006    50
P132    50
Name: Participant, dtype: int64

In [15]:
#only keep participants with 50 rows (disabled and did all gestures)
participant_counts = result['Participant'].value_counts()

# Filter participants who have exactly 50 rows
with_disability = result[result['Participant'].isin(participant_counts[participant_counts == 50].index)].copy()

# Display the filtered dataset
with_disability

,Participant,Gesture_ID,Gesture_Num,feature,Gesture_Encoded
600,P104,air-tap,1,"[[-19.931494507190266], [14.43852161455601], [...",0
601,P104,air-tap,10,"[[-20.473594370607508], [15.84688054857665], [...",0
602,P104,air-tap,2,"[[-19.639110285656656], [14.918420781285667], ...",0
603,P104,air-tap,3,"[[-20.040428885228497], [15.268214541976569], ...",0
604,P104,air-tap,4,"[[-20.05366602050795], [15.511636569909998], [...",0
...,...,...,...,...,...
1110,P132,two-handed-tap,5,"[[-19.07993955504957], [17.24214204325393], [5...",9
1111,P132,two-handed-tap,6,"[[-18.73670013388571], [15.592195296620298], [...",9
1112,P132,two-handed-tap,7,"[[-19.075612774367713], [16.198239062623255], ...",9
1113,P132,two-handed-tap,8,"[[-19.552360489246766], [17.701475682262988], ...",9


CCA functions

In [16]:
# Hold out one participant as the expert
def hold_out_expert(data, participant_column):
    participant_ids = data[participant_column].unique()
    expert_user = np.random.choice(participant_ids)  # Randomly select one participant as expert
    data_expert = data[data[participant_column] == expert_user]  # Expert data
    data_remaining = data[data[participant_column] != expert_user]  # Remaining data
    return data_expert, data_remaining

In [17]:
# Train-test split for remaining data
def split_train_test(data, participant_column, test_size=0.2):
    participant_ids = data[participant_column].unique()
    train_ids, test_ids = train_test_split(participant_ids, test_size=test_size, random_state=42)
    
    train_data = data[data[participant_column].isin(train_ids)]
    test_data = data[data[participant_column].isin(test_ids)]
    
    return train_data, test_data

In [18]:
def flatten_features(feature_column):
    # Convert a column of lists of lists into a 2D NumPy array
    return np.array([np.array(item).flatten() for item in feature_column])
#X_expert = flatten_features(expert_data['feature']) #now  a 2D array with 
#X_other = flatten_features(remaining_data['feature'])

#print(X_other)

In [19]:
#inside k_fold function
# Apply CCA btwn expert and train/test
def apply_cca_between_expert_and_others(expert_data, other_data, target_column, n_components=2):
    # Separate features (X) and target (y)
    
    #after flattening to 2D 
    X_expert = flatten_features(expert_data['feature'])
    X_other = flatten_features(other_data['feature'])

    y_expert = expert_data[target_column]
    y_other = other_data[target_column]
    
    # Apply CCA between expert and other participant
    cca = CCA(n_components=n_components)
    #cca.fit(X_expert, y_expert)
    #print(X_expert.shape)
    #print(X_other.shape)
    cca.fit(X_expert, X_other) #might not run bcuz diff shapes
    #need to figure out how to pass in X_others. if not clear from paper, google cca github and try to find expert user and others
    X_other_cca = cca.transform(X_other)
    
    return X_other_cca, y_other#, X_expert, X_other#, X_other, X_expert, y_expert            think we're good to leave this out?

In [20]:
#inside k_fold function
# Run KNN -- used inside cca but can also be used outside
def run_knn(X_train, y_train, X_test, y_test, n_neighbors=5):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    return accuracy_score(y_test, y_pred)

In [36]:
# Implement k-fold cross-validation and refer to each of the other functions to run CCA between expert user and training and expert user and testing
def k_fold_cross_validation(data, expert_data, participant_column, target_column, k=5, n_neighbors=5, n_components=2):

    group_kf = GroupKFold(n_splits=k)
    accuracies = []
    
    #80/20 train test split. 4 folds in train and 1 in test
    for train_index, test_index in group_kf.split(data, groups=data[participant_column]):
        # Split based on participant IDs
        train_ids = data[participant_column].iloc[train_index].unique()
        test_ids = data[participant_column].iloc[test_index].unique()
        
        train_data = data[data[participant_column].isin(train_ids)]
        test_data = data[data[participant_column].isin(test_ids)]
        
        #print("total data:", data.shape)
        #print("train", train_data[participant_column].unique())
        #print("Test", test_data[participant_column].unique())
        # CCA between expert and training users
        X_train_cca = []
        y_train = []
        for pid in train_ids:
            pid_data = train_data[train_data[participant_column] == pid]
            #print("participant id", pid)
            #print("expert data", expert_data.shape)
            #print("participant data", pid_data.shape)
            X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
            X_train_cca.append(X_pid_cca)
            y_train.append(y_pid)
        
        # Combine all CCA-transformed training data
        X_train_cca = np.vstack(X_train_cca)
        y_train = np.concatenate(y_train)
        
        # CCA between expert and testing users
        X_test_cca = []
        y_test = []
        for pid in test_ids:
            pid_data = test_data[test_data[participant_column] == pid]
            #change to X_expert, X_other for output     X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
            X_test_cca.append(X_pid_cca)
            y_test.append(y_pid)
        
        # Combine all CCA-transformed testing data
        X_test_cca = np.vstack(X_test_cca)
        y_test = np.concatenate(y_test)
        
        # Run KNN on CCA-transformed feature sets
        accuracy = run_knn(X_train_cca, y_train, X_test_cca, y_test, n_neighbors)
        accuracies.append(accuracy)
    
    return accuracies

In [23]:
expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')
expert_data.head()

,Participant,Gesture_ID,Gesture_Num,feature,Gesture_Encoded
1065,P132,air-tap,1,"[[-19.16652871842587], [16.724676482435193], [...",0
1066,P132,air-tap,10,"[[-18.18052481900217], [16.294732335445037], [...",0
1067,P132,air-tap,2,"[[-19.33160301184248], [17.348993130369358], [...",0
1068,P132,air-tap,3,"[[-19.11583968101696], [17.28483517202384], [5...",0
1069,P132,air-tap,4,"[[-18.406120681413714], [16.16664693085155], [...",0


In [35]:
expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')
cca_knn_accuracy=k_fold_cross_validation(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', k=5, n_neighbors=5, n_components=2)
    

total data: (450, 5)
train ['P104' 'P111' 'P114' 'P119' 'P123' 'P124' 'P126']
Test ['P116' 'P132']
total data: (450, 5)
train ['P104' 'P111' 'P116' 'P119' 'P123' 'P124' 'P132']
Test ['P114' 'P126']
total data: (450, 5)
train ['P104' 'P114' 'P116' 'P119' 'P123' 'P126' 'P132']
Test ['P111' 'P124']
total data: (450, 5)
train ['P111' 'P114' 'P116' 'P119' 'P124' 'P126' 'P132']
Test ['P104' 'P123']
total data: (450, 5)
train ['P104' 'P111' 'P114' 'P116' 'P123' 'P124' 'P126' 'P132']
Test ['P119']


In [37]:
#11/11 output with k-fold and averaging for 100 runs

#train_data, test_data = split_train_test(with_disability, 'Participant', test_size=0.2)
#In CCA, the number of components must be less than or equal to the number of features in either dataset or the number of samples minus one, whichever is smaller.
#so changed n_components to 1
standard_rep = {}
for i in range(100):
    expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')
    cca_knn_accuracy=k_fold_cross_validation(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', k=5, n_neighbors=5, n_components=2)
    
    #get mean accuracy for each expert_id
    participants = expert_data['Participant'].unique()
    for participant, accuracy in zip(participants, cca_knn_accuracy):
        if participant not in standard_rep:
            standard_rep[participant] = []
        standard_rep[participant].append(accuracy)


# Calculate mean accuracy per participant
mean_accuracies_per_participant = {participant: np.mean(accuracies) for participant, accuracies in standard_rep.items()}

print("Standardized Mean accuracies per participant:", mean_accuracies_per_participant)


#train_data, test_data = split_train_test(userdef, 'Participant', test_size=0.2)
#In CCA, the number of components must be less than or equal to the number of features in either dataset or the number of samples minus one, whichever is smaller.
#so changed n_components to 1
userdef_rep = {}
for i in range(100):
    expert_data, remaining_data = hold_out_expert(userdef, 'Participant')
    cca_knn_accuracy=k_fold_cross_validation(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', k=5, n_neighbors=5, n_components=2)
    
    #get mean accuracy for each expert_id
    participants = expert_data['Participant'].unique()
    for participant, accuracy in zip(participants, cca_knn_accuracy):
        if participant not in userdef_rep:
            userdef_rep[participant] = []
        userdef_rep[participant].append(accuracy)

# Calculate mean accuracy per participant
mean_accuracies_per_participant = {participant: np.mean(accuracies) for participant, accuracies in userdef_rep.items()}

print("User-Def Mean accuracies per participant:", mean_accuracies_per_participant)




Standardized Mean accuracies per participant: {'P104': 0.6400000000000001, 'P124': 0.6399999999999999, 'P105': 0.58, 'P123': 0.4600000000000001, 'P126': 0.8, 'P114': 0.7, 'P111': 0.7799999999999999, 'P119': 0.66, 'P116': 0.5, 'P132': 0.82}
User-Def Mean accuracies per participant: {'P109': 0.58, 'P108': 0.62, 'P127': 0.6, 'P114': 0.5, 'P132': 0.56, 'P106': 0.6799999999999999, 'P119': 0.5, 'P123': 0.76, 'P005': 0.62, 'P116': 0.48, 'P004': 0.54, 'P118': 0.54, 'P006': 0.54, 'P115': 0.48, 'P103': 0.42, 'P112': 0.4000000000000001, 'P125': 0.6, 'P122': 0.72, 'P111': 0.6, 'P010': 0.56, 'P008': 0.5, 'P121': 0.5, 'P110': 0.6, 'P104': 0.66, 'P107': 0.56, 'P126': 0.54, 'P011': 0.48, 'P102': 0.58}


In [30]:
#11/11 without k-fold
def apply_cca_and_knn(data, expert_data, participant_column, target_column, test_size=0.2, n_neighbors=5, n_components=2):
    # Split the remaining data into train and test sets using the provided function
    train_data, test_data = split_train_test(data, participant_column, test_size=test_size)
    #print("train data shape", train_data.shape)
    #print("test data shape", test_data.shape)

    
    # Apply CCA individually for each training user with the expert
    X_train_cca = []
    y_train = []
    for pid in train_data[participant_column].unique():
        pid_data = train_data[train_data[participant_column] == pid]
        # Individual CCA model and transformation for each training participant
        X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
        X_train_cca.append(X_pid_cca)
        y_train.append(y_pid)
    
    # Combine all CCA-transformed training data
    X_train_cca = np.vstack(X_train_cca)  # Combining individual transformations
    y_train = np.concatenate(y_train)
    
    # Apply CCA individually for each testing user with the expert
    X_test_cca = []
    y_test = []
    for pid in test_data[participant_column].unique():
        pid_data = test_data[test_data[participant_column] == pid]
        # Individual CCA model and transformation for each testing participant
        X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
        X_test_cca.append(X_pid_cca)
        y_test.append(y_pid)
    
    # Combine all CCA-transformed testing data
    X_test_cca = np.vstack(X_test_cca)  # Combining individual transformations
    y_test = np.concatenate(y_test)
    
    # Run KNN on CCA-transformed feature sets
    accuracy = run_knn(X_train_cca, y_train, X_test_cca, y_test, n_neighbors)
    
    return accuracy
#expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')
#In CCA, the number of components must be less than or equal to the number of features in either dataset or the number of samples minus one, whichever is smaller.
#so changed n_components to 1
#cca_knn_accuracy=apply_cca_and_knn(remaining_data, expert_data, 'Participant', 'Gesture_Encoded')
#

In [37]:
#running cca with no k-fold and averaging for 100 runs
standard_nokfold = {}  # Dictionary to store accuracies per participant

for i in range(100):
    expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')

    
    # Apply CCA and KNN, assuming 'apply_cca_and_knn' returns accuracy for this iteration
    accuracy = apply_cca_and_knn(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', n_components=1)
    
    # Add the accuracy to the corresponding participant's list
    for pid in expert_data['Participant'].unique():
        if pid not in standard_nokfold:
            standard_nokfold[pid] = []
            standard_nokfold[pid].append(accuracy)

# Calculate the mean accuracy for each participant across all iterations
mean_accuracies_per_participant = {pid: np.mean(accs) for pid, accs in standard_nokfold.items()}

print(" No KNN Standardized Mean accuracies per participant:", mean_accuracies_per_participant)




userdef_nokfold = {}
for i in range(100):
    expert_data, remaining_data = hold_out_expert(userdef, 'Participant')
#train_data, test_data = split_train_test(remaining_data, 'Participant', test_size=0.2)
#In CCA, the number of components must be less than or equal to the number of features in either dataset or the number of samples minus one, whichever is smaller.
#so changed n_components to 1
    accuracy=apply_cca_and_knn(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', n_neighbors=5, n_components=2)
    #get mean accuracy for each expert_id
    for pid in expert_data['Participant'].unique():
        if pid not in userdef_nokfold:
            userdef_nokfold[pid] = []
            userdef_nokfold[pid].append(accuracy)

mean_accuracies_per_participant = {pid: np.mean(accs) for pid, accs in userdef_nokfold.items()}

print("No KNN User-Def Mean accuracies per participant:", mean_accuracies_per_participant)

#Lower accuracy without k-fold

 No KNN Standardized Mean accuracies per participant: {'P114': 0.15, 'P123': 0.23, 'P111': 0.11, 'P105': 0.31, 'P119': 0.18, 'P116': 0.15, 'P124': 0.25, 'P132': 0.28, 'P126': 0.17, 'P104': 0.18}
No KNN User-Def Mean accuracies per participant: {'P107': 0.15666666666666668, 'P102': 0.18666666666666668, 'P122': 0.22, 'P008': 0.16333333333333333, 'P128': 0.21, 'P121': 0.16, 'P132': 0.17666666666666667, 'P010': 0.19666666666666666, 'P109': 0.12333333333333334, 'P110': 0.21, 'P112': 0.23666666666666666, 'P011': 0.19666666666666666, 'P115': 0.20333333333333334, 'P105': 0.22333333333333333, 'P116': 0.21333333333333335, 'P005': 0.17, 'P123': 0.18666666666666668, 'P006': 0.22666666666666666, 'P127': 0.24, 'P125': 0.17333333333333334, 'P108': 0.21333333333333335, 'P111': 0.24, 'P004': 0.16666666666666666, 'P103': 0.20333333333333334, 'P104': 0.25666666666666665, 'P118': 0.22666666666666666, 'P114': 0.23, 'P124': 0.18333333333333332, 'P106': 0.15333333333333332, 'P119': 0.19666666666666666}


In [22]:
#baseline PCA
def run_pca_knn(df, n_components=8, n_neighbors=5, test_size=0.2):
    """
    Perform PCA on the 'feature' column and KNN classification on the transformed data.

    Parameters:
        df (DataFrame): Input DataFrame containing 'feature' and 'Gesture_ID' columns.
        n_components (int): Number of PCA components.
        n_neighbors (int): Number of neighbors for KNN.
        test_size (float): Fraction of data to use as the test set.
        random_state (int): Random state for train-test split.

    Returns:
        explained_variance_ratio (array): Explained variance ratio of PCA components.
        accuracy (float): Classification accuracy of KNN on the test set.
    """
    # Convert the 'feature' column from nested lists to a flat list for each entry
    df['feature_flat'] = df['feature'].apply(lambda x: np.ravel(x))

    # Extract features and labels
    X = np.vstack(df['feature_flat'].values)
    y = df['Gesture_ID']

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Apply PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=test_size)

    # Train KNN
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return pca.explained_variance_ratio_, accuracy

# Example usage:
# df is your DataFrame with 'feature' and 'Gesture_ID' columns
print("standardized")
explained_variance_ratio, accuracy = run_pca_knn(with_disability, n_components=10, n_neighbors=5)
print("PCA Explained Variance Ratio:", explained_variance_ratio)
print("KNN Classification Accuracy:", accuracy)
print("")
print("user-def") #better at user-def than standardized
explained_variance_ratio, accuracy = run_pca_knn(userdef, n_components=10, n_neighbors=5)
print("PCA Explained Variance Ratio:", explained_variance_ratio)
print("KNN Classification Accuracy:", accuracy)

standardized
PCA Explained Variance Ratio: [0.30845778 0.13776652 0.10729692 0.06860659 0.05323273 0.03708053
 0.03274266 0.02663575 0.02342023 0.02088873]
KNN Classification Accuracy: 0.86

user-def
PCA Explained Variance Ratio: [0.28213244 0.14262123 0.08663335 0.05707593 0.0444905  0.04006048
 0.03554242 0.02957199 0.02531161 0.02334016]
KNN Classification Accuracy: 0.9225806451612903


In [49]:
#baseline knn
def run_knn_baseline_with_expert(expert_data, remaining_data, target_column, n_neighbors=5, k=5):
    # Flatten the features from both datasets
    X_expert = flatten_features(expert_data['feature'])
    X_remaining = flatten_features(remaining_data['feature'])
    
    # Combine the feature sets and their corresponding targets
    X_combined = np.vstack((X_expert, X_remaining))
    y_expert = expert_data[target_column].values
    y_remaining = remaining_data[target_column].values
    y_combined = np.concatenate((y_expert, y_remaining))

    
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = []
    
    for train_index, test_index in kf.split(X_combined):
        X_train, X_test = X_combined[train_index], X_combined[test_index]
        y_train, y_test = y_combined[train_index], y_combined[test_index]

        # Run KNN on the combined dataset
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    return accuracies
expert_data, remaining_data = hold_out_expert(with_disability, 'Participant')
baseline_accuracies = run_knn_baseline_with_expert(expert_data, remaining_data, 'Gesture_Encoded')
print(f'KNN Accuracy: {baseline_accuracies}')
print("standardized", np.mean(baseline_accuracies))

pers_expert_data, pers_remaining_data = hold_out_expert(userdef, 'Participant')
baseline_accuracies = run_knn_baseline_with_expert(pers_expert_data, pers_remaining_data, 'Gesture_Encoded')
print(f'KNN Accuracy: {baseline_accuracies}')
print("userdef", np.mean(baseline_accuracies))

KNN Accuracy: [0.9, 0.96, 0.97, 0.93, 0.96]
standardized 0.9440000000000002
KNN Accuracy: [0.9516129032258065, 0.9483870967741935, 0.9516129032258065, 0.9483870967741935, 0.9516129032258065]
userdef 0.9503225806451614


## NOT USING BELOW

In [41]:
#see the shapes of all pid and expert_data for fitting
def k_fold_cross_validation(data, expert_data, participant_column, target_column, k=5, n_neighbors=5, n_components=2):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    for train_index, test_index in kf.split(data[participant_column].unique()):
        train_ids = data[participant_column].unique()[train_index]
        test_ids = data[participant_column].unique()[test_index]
        print(train_ids.shape)
        print(test_ids.shape)

        train_data = data[data[participant_column].isin(train_ids)]
        test_data = data[data[participant_column].isin(test_ids)]
        diff_shape = []

        for pid in train_ids:
            pid_data = train_data[train_data[participant_column] == pid]
            if expert_data.shape != pid_data.shape:
                diff_shape.append(pid)
    return diff_shape
k_fold_cross_validation(remaining_data, expert_data, 'Participant', 'Gesture_Encoded', k=5, n_neighbors=5, n_components=1)

#shape problem with 'P104', 'P111', 'P114', 'P116', 'P119', 'P123', 'P126', 'P131', 'P132'

(12,)
(4,)
(13,)
(3,)
(13,)
(3,)
(13,)
(3,)
(13,)
(3,)


['P104', 'P111', 'P114', 'P116', 'P119', 'P123', 'P126', 'P131', 'P132']

In [38]:
np.mean(cca_knn_accuracy) #accuracy for each k-fold.... really poor (maybe because n_components is 1??)

0.644

In [42]:
#### don't use
# Hold out one participant as the expert
def hold_out_expert(data, participant_column):
    participant_ids = data[participant_column].unique()
    expert_user = np.random.choice(participant_ids)  # Randomly select one participant as expert
    data_expert = data[data[participant_column] == expert_user]  # Expert data
    data_remaining = data[data[participant_column] != expert_user]  # Remaining data
    return data_expert, data_remaining

def apply_cca_between_expert_and_others(expert_data, participant_data, target_column, n_components):
    """
    Apply CCA between expert data and a participant's data.
    
    Parameters:
        expert_data (DataFrame): Expert user data.
        participant_data (DataFrame): Participant data.
        target_column (str): Name of the target column.
        n_components (int): Number of CCA components.
    
    Returns:
        X_cca (array): CCA-transformed features for the participant data.
        y (array): Target values for the participant data.
    """
    # Extract features and target
    X_expert = np.vstack(expert_data['feature'].apply(lambda x: np.ravel(x)).values)
    X_participant = np.vstack(participant_data['feature'].apply(lambda x: np.ravel(x)).values)
    y = participant_data[target_column].values

    # Encode target labels for CCA
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y).reshape(-1, 1)

    # Apply CCA
    cca = CCA(n_components=n_components)
    X_cca, _ = cca.fit_transform(X_participant, y_encoded)

    return X_cca, y

def run_knn(X_train, y_train, X_test, y_test, n_neighbors):
    """
    Run KNN classification on the transformed data and return the accuracy.
    
    Parameters:
        X_train (array): CCA-transformed training features.
        y_train (array): Training labels.
        X_test (array): CCA-transformed test features.
        y_test (array): Test labels.
        n_neighbors (int): Number of neighbors for KNN.
    
    Returns:
        accuracy (float): Classification accuracy of KNN.
    """
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    return accuracy_score(y_test, y_pred)

def k_fold_cross_validation(data, expert_data, participant_column, target_column, k=5, n_neighbors=5, n_components=2):
    """
    Perform k-fold cross-validation using CCA between expert user data and participants,
    and KNN classification on the transformed data.
    
    Parameters:
        data (DataFrame): Input DataFrame containing participants' data.
        expert_data (DataFrame): Expert user data for CCA.
        participant_column (str): Name of the participant column.
        target_column (str): Name of the target column.
        k (int): Number of folds for cross-validation.
        n_neighbors (int): Number of neighbors for KNN.
        n_components (int): Number of CCA components.
    
    Returns:
        accuracies (list): List of accuracies for each fold.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = []
    
    # k-fold cross-validation
    for train_index, test_index in kf.split(data[participant_column].unique()):
        train_ids = data[participant_column].unique()[train_index]
        test_ids = data[participant_column].unique()[test_index]
        
        train_data = data[data[participant_column].isin(train_ids)]
        test_data = data[data[participant_column].isin(test_ids)]
        
        # Apply CCA between expert and training participants
        X_train_cca = []
        y_train = []
        for pid in train_ids:
            pid_data = train_data[train_data[participant_column] == pid]
            X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
            X_train_cca.append(X_pid_cca)
            y_train.append(y_pid)
        
        # Combine CCA-transformed training data
        X_train_cca = np.vstack(X_train_cca)
        y_train = np.concatenate(y_train)
        
        # Apply CCA between expert and testing participants
        X_test_cca = []
        y_test = []
        for pid in test_ids:
            pid_data = test_data[test_data[participant_column] == pid]
            X_pid_cca, y_pid = apply_cca_between_expert_and_others(expert_data, pid_data, target_column, n_components)
            X_test_cca.append(X_pid_cca)
            y_test.append(y_pid)
        
        # Combine CCA-transformed testing data
        X_test_cca = np.vstack(X_test_cca)
        y_test = np.concatenate(y_test)
        
        # Run KNN on CCA-transformed data
        accuracy = run_knn(X_train_cca, y_train, X_test_cca, y_test, n_neighbors)
        accuracies.append(accuracy)
    
    return accuracies
expert_data, remaining_data = hold_out_expert(result, 'Participant')
accuracies = k_fold_cross_validation(remaining_data, expert_data, 'Participant', 'Gesture_ID', k=5, n_neighbors=10, n_components=1)
print("Cross-validation accuracies:", accuracies)
print("Average accuracy:", np.mean(accuracies))

Cross-validation accuracies: [0.3433962264150943, 0.4666666666666667, 0.38, 0.255, 0.31]
Average accuracy: 0.3510125786163522


In [44]:
#solution with resampling -- didn't use for final run through
from sklearn.cross_decomposition import CCA
from sklearn.utils import resample

def apply_cca_between_expert_and_others(expert_data, other_data, target_column, n_components=2):
    # Separate features (X) and target (y)
    X_expert = flatten_features(expert_data['feature'])
    X_other = flatten_features(other_data['feature'])

    y_expert = expert_data[target_column]
    y_other = other_data[target_column]
    
    # Match the sample size between X_expert and X_other
    X_other_resampled, y_other_resampled = resample(X_other, y_other, n_samples=X_expert.shape[0], random_state=42)
    
    # Apply CCA between expert and other participant
    cca = CCA(n_components=n_components)
    cca.fit(X_expert, X_other_resampled)
    X_other_cca = cca.transform(X_other_resampled)
    
    return X_other_cca, y_other_resampled#, X_expert, X_other_resampled




In [ ]:
#need to relabel freq_data to time data. and turn that to np.fft.fft to convert to time

# Parseval's theorem verification function
def parsevals_theorem_verification(df_freq):
    time_domain_data = {}
    freq_domain_squares_sum = {}
    time_domain_squares_sum = {}

    for sensor in df_freq.columns:
        # Extract the time domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        
        # FFT to transform to freq domain
        freq_data = np.fft.fft(time_data)
        
        # Parseval's theorem: Sum of squares in the frequency domain
        freq_domain_sum_sq = np.sum(np.abs(freq_data)**2)
        
        # Parseval's theorem: Sum of squares in the time domain
        time_domain_sum_sq = np.sum(np.abs(time_data)**2)
        
        # Store results in dictionaries
        time_domain_data[sensor] = time_data
        freq_domain_squares_sum[sensor] = freq_domain_sum_sq
        time_domain_squares_sum[sensor] = time_domain_sum_sq
        
        # Output verification for this sensor
        print(f'Sensor: {sensor}')
        print(f'Sum of squares (Frequency domain): {freq_domain_sum_sq}')
        print(f'Sum of squares (Time domain): {time_domain_sum_sq}')
        print(f'Parseval\'s theorem holds: {np.isclose(freq_domain_sum_sq, time_domain_sum_sq)}\n')
    
    return pd.DataFrame(time_domain_data), freq_domain_squares_sum, time_domain_squares_sum

# Assuming df_freq is your dataframe with frequency domain EMG data
# df_time, freq_squares, time_squares = parsevals_theorem_verification(df_freq)


#want to see that freq_domain_sum_sq is the same as time_domain_sum_sq
#can try and generalize by gesture trial as opposed to by column
#load in metadata and break up huge dataframe by each gesture chunk

In [ ]:
parsevals_theorem_verification(EMG_sliced)

In [ ]:
###NOT USING
#RAN INTO ISSUE WITH TAKING SECOND LOG OF NEG VALUES SINCE LOG(ZERO/DOT) GAVE NEG VALUES
# either follow code in cell below to take maximum or maybe do abs value of log(zero/dot)?? these are both diff from original formula though
def third_order_moment_log(df_freq, second_order_moments_raw, first_order_moments_raw, zero_order_moments_raw):
    # Step 1: Compute the square roots
    sqrt_first_diff = np.sqrt(zero_order_moments_raw - first_order_moments_raw)
    sqrt_second_diff = np.sqrt(zero_order_moments_raw - second_order_moments_raw)
    #print(sqrt_first_diff)
    #print(sqrt_second_diff)
    
    # Step 2: Perform the dot product
    dot_product = np.dot(sqrt_first_diff, sqrt_second_diff)
    #print(dot_product)
    # Step 3: Compute the sparseness formula
    sparseness = np.log(np.abs(zero_order_moments_raw / dot_product))
    print(zero_order_moments_raw)
    print(dot_product)
    
    print(sparseness)
    
    # Step 4: Apply the logarithm again for third-order moments
    third_order_moments_log = np.log(sparseness)
    
    return third_order_moments_log
third_order_moments_log = third_order_moment_log(EMG_sliced, second_order_moments_raw, first_order_moments_raw, zero_order_moments_raw)

In [ ]:
###NOT USING
def freq_second_order_moment_log(df_freq, zero_order_moments_raw):
    second_order_moments_log = []
    second_order_moments_raw = []

    for i, sensor in enumerate(df_freq.columns):
        # Extract the frequency domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        freq_data = np.fft.fft(time_data)
        
        # Step 1: Take first derivative
        first_deriv = np.gradient(freq_data)
        
        #Step 2: take second derivative
        second_deriv = np.gradient(first_deriv)
        
        # Step 3: Square the signal (power) at each frequency of the first deriv
        signal_squared = np.abs(second_deriv) ** 2
        
        # Step 4: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 4: Take the logarithm of the total power
        log_total_power = np.log(total_power/(zero_order_moments_raw[i]**4))
        
        # Store the result in the dictionary
        second_order_moments_log.append(log_total_power)
        second_order_moments_raw.append(total_power)

    
    return second_order_moments_log, second_order_moments_raw

freq_second_order_moments_log, second_order_moments_raw = freq_first_order_moment_log(EMG_sliced, zero_order_moments_raw)
#output = zero_order_moments_log is the zeroeth feature

In [ ]:
###NOT USING
def freq_first_order_moment_log(df_freq, freq_zero_order_moments_raw):
    # Initialize lists to store the results for each sensor
    first_order_moments_log = []
    first_order_moments_raw = []
    
    for i,sensor in enumerate(df_freq.columns):
        # Extract the frequency domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        freq_data = np.fft.fft(time_data)
        
        # Step 1: Take first derivative
        first_deriv = np.gradient(freq_data)
        
        # Step 2: Square the signal (power) at each frequency of the first deriv
        signal_squared = np.abs(first_deriv) ** 2
        
        # Step 3: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 4: Take the logarithm of the total power
        log_total_power = np.log(total_power / (zero_order_moments_raw[i]**2))
        
        # Store the results in the lists
        first_order_moments_log.append(log_total_power)
        first_order_moments_raw.append(total_power)

    # Convert lists to numpy arrays for consistency
    first_order_moments_log = np.array(first_order_moments_log)
    first_order_moments_raw = np.array(first_order_moments_raw)
    
    return first_order_moments_log, first_order_moments_raw

# Example usage:
freq_first_order_moments_log, freq_first_order_moments_raw = freq_first_order_moment_log(EMG_sliced, freq_zero_order_moments_raw)

In [ ]:
###NOT USING
def freq_zero_order_moment_log(df_freq):
    zero_order_moments_log = []
    zero_order_moments_raw = []

    for sensor in df_freq.columns:
        # Extract the frequency domain data for the sensor (column of the dataframe)
        time_data = df_freq[sensor].values
        freq_data = np.fft.fft(time_data)

        
        # Step 1: Square the signal (power) at each frequency
        signal_squared = np.abs(freq_data) ** 2
        
        # Step 2: Integrate (sum) the squared values to get the total power
        total_power = np.sum(signal_squared)
        
        # Step 3: Take the logarithm of the total power
        log_total_power = np.log(total_power)
        
        # Store the result in the dictionary
        zero_order_moments_log.append(log_total_power)
        zero_order_moments_raw.append(total_power)
        
        #convert to np.array for consistency
    
    return zero_order_moments_log, zero_order_moments_raw

freq_zero_order_moments_log, freq_zero_order_moments_raw = freq_zero_order_moment_log(EMG_sliced)
#output = zero_order_moments_log is the zeroeth feature

In [ ]:
#to do fft: need to slice out EMG data and only use that
EMG_sliced = EMG_stand.iloc[:,3:]
EMG_sliced